# GRPO Training for Italian Exercise Generator

## Setup

In [1]:
# --- Cell 1: Setup and Imports ---

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Navigate to your project directory
# Make sure this path is correct for your Google Drive setup
import os
project_path = '/content/drive/MyDrive/Colab Notebooks/italian_teacher'
os.chdir(project_path)
print(f"Changed directory to: {os.getcwd()}")

# Install dependencies
!pip install -q transformers trl accelerate peft datasets spacy sentence-transformers bitsandbytes json5 openai tqdm nest_asyncio
!python -m spacy download it_core_news_sm

# Standard library imports
import json
import random
from getpass import getpass

# Third-party imports
import torch
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import GRPOConfig, GRPOTrainer

# Local module imports
from src.rl.multi_reward_async import create_async_multi_reward
from src.rl.prompt_formatter import format_prompt_with_chat_template
from src.rl.reward_function import ExerciseRewardFunction

# Environment setup
os.environ["WANDB_DISABLED"] = "true"

print("\n--- Environment Setup ---")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Changed directory to: /content/drive/MyDrive/Colab Notebooks/italian_teacher
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 34.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.

--- Environment Setup ---
PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: NVIDIA A100-SXM4-80GB


In [2]:
# --- Cell 2: Configuration ---
# All training parameters are here for easy modification.

# --- General Configuration ---
USE_OPENAI = True
BASE_MODEL_PATH = "./models/italian_v8_grpo_round2"  # Input model for this training run
OUTPUT_DIR = "./models/TeacherPet_italian_grpo"           # Where the new model will be saved
NUM_SAMPLES = 2000                                # Number of training requests to use
RANDOM_SEED = 44                                  # Seed for reproducibility
DISABLED_SCORERS = ["fluency"]                    # List of scorers to disable

# --- GRPO Configuration ---
# This uses the same optimized settings you provided, with corrections.
grpo_config = GRPOConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,

    # CORRECTED: A lower learning rate for stable fine-tuning.
    learning_rate=5e-6,  # 5e-6 is lower than the previous 9e-6.

    warmup_steps=50,
    logging_steps=5,
    save_steps=100,
    save_total_limit=3,
    bf16=True,
    remove_unused_columns=False,
    report_to="none",

    # GRPO-specific generation settings
    num_generations=4,
    max_completion_length=350,  # This is the primary controller for max new tokens.
    temperature=0.7,
    generation_batch_size=32,

    # Cleaned-up generation_kwargs
    generation_kwargs={
      "bos_token_id": 128000,
      "do_sample": True,
      # CORRECTED: Removed duplicate eos_token_id
      "eos_token_id": [128009, 128001],
      "pad_token_id": 128009,
      # REMOVED: temperature and max_new_tokens are controlled by top-level args.
      "top_p": 0.9,
      "padding_side": "left"
    }
)

print("✅ Configuration loaded and corrected.")


✅ Configuration loaded and corrected.


In [3]:
# --- Cell 3: Helper Functions ---

def configure_openai(use_openai: bool):
    """Configures the OpenAI API key if enabled."""
    if not use_openai:
        print("ℹ️ OpenAI usage is disabled. LLM-based scorers will be skipped.")
        return

    api_key = os.environ.get("OPENAI_API_KEY")
    if not api_key:
        print("OpenAI API key not found in environment variables.")
        try:
            api_key = getpass("Enter your OpenAI API key: ")
        except (IOError, EOFError):
            print("Could not read API key. Disabling OpenAI usage.")
            return

    if api_key:
        os.environ["OPENAI_API_KEY"] = api_key
        print("✅ OpenAI API key configured.")
    else:
        print("⚠️ No OpenAI API key provided. LLM-based scorers will be skipped.")

def load_training_data(tokenizer, num_samples: int, seed: int):
    """Loads or generates training requests and prepares the dataset."""
    requests_path = "src/rl/training_requests.json"
    if os.path.exists(requests_path):
        print(f"Loading existing training requests from {requests_path}...")
        with open(requests_path, "r") as f:
            training_requests = json.load(f)
    else:
        from src.rl.generate_training_requests import generate_training_requests
        print(f"Generating {num_samples} new training requests...")
        training_requests = generate_training_requests(
            num_requests=num_samples,
            output_path=requests_path
        )

    print(f"✅ Loaded {len(training_requests)} total training requests.")

    prompts = [
        format_prompt_with_chat_template(req, tokenizer, add_examples=True)
        for req in training_requests
    ]

    if len(prompts) > num_samples:
        print(f"Sampling {num_samples} requests for this run (seed={seed})...")
        random.seed(seed)
        random_indices = random.sample(range(len(prompts)), num_samples)
        prompts = [prompts[i] for i in random_indices]
        training_requests_subset = [training_requests[i] for i in random_indices]
    else:
        training_requests_subset = training_requests

    return Dataset.from_dict({
        "prompt": prompts,
        "request": training_requests_subset,
    })

print("✅ Helper functions defined.")


✅ Helper functions defined.


In [4]:
# --- Cell 4: Main Execution ---

print("=" * 80)
print("🚀 STARTING GRPO TRAINING RUN")
print("=" * 80)

# 1. Configure OpenAI
configure_openai(USE_OPENAI)

# 2. Load Model and Tokenizer
print(f"\nLoading base model from: {BASE_MODEL_PATH}")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH, padding_side='left')
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"



model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_PATH,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    use_cache=False,
)
model.gradient_checkpointing_enable()
print("✅ Model and tokenizer loaded.")
# CRITICAL FIX: Synchronize the model's config with the tokenizer's settings.
# This ensures the model's `generate` method uses the correct padding side.
model.config.pad_token_id = tokenizer.pad_token_id
model.config.padding_side = tokenizer.padding_side

# 3. Prepare Dataset and Reward Function
train_dataset = load_training_data(tokenizer, num_samples=NUM_SAMPLES, seed=RANDOM_SEED)

print("\nInitializing reward function...")
# Pass the concurrency limit directly to the main reward function
reward_fn_instance = ExerciseRewardFunction(
    device="cuda",
    disabled_scorers=DISABLED_SCORERS,
    concurrency_limit=20  # Explicitly set concurrency here
)
# The wrapper is now much simpler
reward_func = create_async_multi_reward(
    reward_fn_instance,
    use_openai=USE_OPENAI
)
print("✅ Reward function ready.")

# 4. Initialize Trainer
print("\nInitializing GRPO Trainer...")
trainer = GRPOTrainer(
    model=model,
    args=grpo_config,
    reward_funcs=reward_func,
    train_dataset=train_dataset,
    processing_class=tokenizer,
)
print("✅ GRPO Trainer initialized. Ready to start training!")

# 5. Start Training
print("\n" + "=" * 80)
print("🔥 Training begins! 🔥")
print("=" * 80)
trainer.train()
print("\n" + "=" * 80)
print("🎉 Training complete! 🎉")
print("=" * 80)

# 6. Save Final Model
print(f"\nSaving final model to {OUTPUT_DIR}...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("✅ Model and tokenizer saved successfully.")


🚀 STARTING GRPO TRAINING RUN
OpenAI API key not found in environment variables.
Enter your OpenAI API key: ··········
✅ OpenAI API key configured.

Loading base model from: ./models/italian_v8_grpo_round2


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Model and tokenizer loaded.
Loading existing training requests from src/rl/training_requests.json...
✅ Loaded 2000 total training requests.

Initializing reward function...
Loading spaCy model: it_core_news_sm...
✅ spaCy model loaded
Reward function will use device: cuda
Initializing scorers...
  ✅ LLM scoring enabled for cefr_alignment (batch size: 10)
  ✅ LLM scoring enabled for grammar_correctness (batch size: 10)
  ✅ LLM scoring enabled for coherence (batch size: 10)
Loading sentence transformer for topic similarity...


The model is already on multiple devices. Skipping the move to device specified in `args`.


...


🎯 Reward calculation complete (116.6s):
   Grammar   : min=36.0, max=62.5, avg=50.5
   Coherence : min=50.0, max=90.0, avg=57.0
   Topic     : min=27.5, max=100.0, avg=79.3
   Quality   : min=42.5, max=100.0, avg=72.0
   Diversity : min=80.0, max=100.0, avg=97.8
   TOTAL     : min=0.000, max=73.000, avg=47.783


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



⏳ Step 1/3: Parsing 32 JSON completions...


⏳ Step 2/3: Scoring 32 completions with batched reward function...


KeyboardInterrupt: 